In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import librosa
import os
import joblib

In [2]:
# Định nghĩa hàm ánh xạ valence và arousal sang nhãn cảm xúc
def map_to_emotion(valence, arousal):
    if valence > 5 and arousal > 5:
        return 'HAPPY'
    elif valence < 5 and arousal < 5:
        return 'SAD'
    elif valence > 5 and arousal > 7:
        return 'ENERGY'
    elif valence > 5 and 3 < arousal < 7:
        return 'ROMANTIC'
    elif 3 < valence < 7 and arousal < 3:
        return 'CHILL'
    else:
        return 'OTHER'

In [3]:
# Hàm trích xuất đặc trưng từ file MP3
def extract_features_from_mp3(mp3_path):
    try:
        y, sr = librosa.load(mp3_path, sr=None)
        mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)  # 13 đặc trưng
        chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)     # 12 đặc trưng
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr), axis=1)  # 7 đặc trưng
        tempo = librosa.beat.tempo(y=y, sr=sr)[0]                         # 1 đặc trưng
        rms = np.mean(librosa.feature.rms(y=y))                            # 1 đặc trưng
        zcr = np.mean(librosa.feature.zero_crossing_rate(y=y))            # 1 đặc trưng
        features = np.concatenate((mfcc, chroma, spectral_contrast, [tempo, rms, zcr]))
        return features  # Tổng cộng 35 đặc trưng
    except Exception as e:
        print(f"Error processing {mp3_path}: {e}")
        return None

In [ ]:
# Chuẩn bị dữ liệu huấn luyện
def prepare_training_data(annotation_path, audio_dir, feature_save_dir="/media/haphuthinh/Data/Workspace/UIT/DO_AN_2/MusicEmotionDetection/TrainedModel/random_forest_regressor"):
    # Tạo thư mục lưu đặc trưng nếu chưa tồn tại
    if not os.path.exists(feature_save_dir):
        os.makedirs(feature_save_dir)
    
    # Đường dẫn file lưu đặc trưng
    features_file = os.path.join(feature_save_dir, "features.npy")
    valence_file = os.path.join(feature_save_dir, "valence.npy")
    arousal_file = os.path.join(feature_save_dir, "arousal.npy")

    # Kiểm tra xem các file đặc trưng đã tồn tại chưa
    if (os.path.exists(features_file) and 
        os.path.exists(valence_file) and 
        os.path.exists(arousal_file)):
        print("Tải các đặc trưng đã lưu trước đó...")
        X = np.load(features_file)
        y_valence = np.load(valence_file)
        y_arousal = np.load(arousal_file)
        print(f"Đã tải: {X.shape[0]} mẫu")
        return X, y_valence, y_arousal

    # Nếu chưa có file đặc trưng, tiến hành trích xuất
    annotations = pd.read_csv(annotation_path)
    annotations.columns = annotations.columns.str.strip()

    X = []
    y_valence = []
    y_arousal = []

    for index, row in annotations.iterrows():
        if (index + 1) % 10 == 0:
            print(f"Đang xử lý bài hát {index + 1}/{len(annotations)}: {row['song_id']}")

        song_id = row['song_id']
        mp3_path = os.path.join(audio_dir, f"{int(song_id)}.mp3")
        if os.path.exists(mp3_path):
            features = extract_features_from_mp3(mp3_path)
            if features is not None:
                X.append(features)
                y_valence.append(row['valence_mean'])
                y_arousal.append(row['arousal_mean'])
        else:
            print(f"File not found: {mp3_path}")

    # Chuyển thành numpy array
    X = np.array(X)
    y_valence = np.array(y_valence)
    y_arousal = np.array(y_arousal)

    # Lưu đặc trưng
    print("Lưu đặc trưng đã trích xuất...")
    np.save(features_file, X)
    np.save(valence_file, y_valence)
    np.save(arousal_file, y_arousal)
    
    print(f"Đã lưu: {X.shape[0]} mẫu")
    return X, y_valence, y_arousal

In [ ]:
# Huấn luyện mô hình
def train_model(X, y_valence, y_arousal):
    # Chia dữ liệu thành tập huấn luyện và kiểm tra
    X_train, X_test, y_valence_train, y_valence_test = train_test_split(X, y_valence, test_size=0.2, random_state=42)
    _, _, y_arousal_train, y_arousal_test = train_test_split(X, y_arousal, test_size=0.2, random_state=42)

    print(f"Kích thước tập huấn luyện: {X_train.shape}, Kích thước tập kiểm tra: {X_test.shape}")

    # Chuẩn hóa đặc trưng
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Huấn luyện mô hình Random Forest Regressor cho valence và arousal
    model_valence = RandomForestRegressor(n_estimators=100, random_state=42)
    model_valence.fit(X_train, y_valence_train)

    model_arousal = RandomForestRegressor(n_estimators=100, random_state=42)
    model_arousal.fit(X_train, y_arousal_train)

    # Đánh giá mô hình
    y_valence_pred = model_valence.predict(X_test)
    y_arousal_pred = model_arousal.predict(X_test)
    print(f"Valence MSE: {mean_squared_error(y_valence_test, y_valence_pred)}")
    print(f"Arousal MSE: {mean_squared_error(y_arousal_test, y_arousal_pred)}")

    return model_valence, model_arousal, scaler

In [6]:
# Hàm dự đoán valence, arousal và chuyển thành emotion
def predict_emotion(mp3_path, model_valence, model_arousal, scaler):
    features = extract_features_from_mp3(mp3_path)
    if features is not None:
        features = scaler.transform([features])  # Chuẩn hóa đặc trưng
        valence = model_valence.predict(features)[0]
        arousal = model_arousal.predict(features)[0]
        emotion = map_to_emotion(valence, arousal)
        return emotion, valence, arousal
    else:
        return "Error", None, None

In [7]:
# Thực thi chương trình
if __name__ == "__main__":
    # Đường dẫn đến file annotation và thư mục chứa file MP3
    annotation_path = '/media/haphuthinh/Data/Workspace/UIT/DO_AN_2/MusicEmotionDetection/Dataset/DEAM/annotations/annotations averaged per song/song_level/static_annotations_averaged_songs_1_2000.csv'
    audio_dir = '/media/haphuthinh/Data/Workspace/UIT/DO_AN_2/MusicEmotionDetection/Dataset/DEAM/MEMD_audio'

    # Bước 1: Chuẩn bị dữ liệu và huấn luyện mô hình
    print("Đang chuẩn bị dữ liệu huấn luyện...")
    X, y_valence, y_arousal = prepare_training_data(annotation_path, audio_dir)
    if len(X) == 0 or len(y_valence) == 0 or len(y_arousal) == 0:
        print("Không tải được dữ liệu. Vui lòng kiểm tra thư mục và file dữ liệu.")
    else:
        print("Đang huấn luyện mô hình...")
        model_valence, model_arousal, scaler = train_model(X, y_valence, y_arousal)

        joblib.dump(model_valence, '/media/haphuthinh/Data/Workspace/UIT/DO_AN_2/MusicEmotionDetection/TrainedModel/random_forest_regressor/random_forest_valence_model.pkl')
        joblib.dump(model_arousal, '/media/haphuthinh/Data/Workspace/UIT/DO_AN_2/MusicEmotionDetection/TrainedModel/random_forest_regressor/random_forest_arousal_model.pkl')
        joblib.dump(scaler, '/media/haphuthinh/Data/Workspace/UIT/DO_AN_2/MusicEmotionDetection/TrainedModel/random_forest_regressor/random_forest_scaler.pkl')

Đang chuẩn bị dữ liệu huấn luyện...


/tmp/ipykernel_22193/1900758238.py:8: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(y=y, sr=sr)[0]                         # 1 đặc trưng


Đang xử lý bài hát 10/1744: 17.0
Đang xử lý bài hát 20/1744: 35.0
Đang xử lý bài hát 30/1744: 47.0
Đang xử lý bài hát 40/1744: 58.0
Đang xử lý bài hát 50/1744: 71.0
Đang xử lý bài hát 60/1744: 82.0
Đang xử lý bài hát 70/1744: 92.0
Đang xử lý bài hát 80/1744: 105.0
Đang xử lý bài hát 90/1744: 115.0
Đang xử lý bài hát 100/1744: 125.0
Đang xử lý bài hát 110/1744: 136.0
Đang xử lý bài hát 120/1744: 149.0
Đang xử lý bài hát 130/1744: 159.0
Đang xử lý bài hát 140/1744: 170.0
Đang xử lý bài hát 150/1744: 180.0
Đang xử lý bài hát 160/1744: 191.0
Đang xử lý bài hát 170/1744: 202.0
Đang xử lý bài hát 180/1744: 212.0
Đang xử lý bài hát 190/1744: 222.0
Đang xử lý bài hát 200/1744: 233.0
Đang xử lý bài hát 210/1744: 243.0
Đang xử lý bài hát 220/1744: 258.0
Đang xử lý bài hát 230/1744: 274.0
Đang xử lý bài hát 240/1744: 286.0
Đang xử lý bài hát 250/1744: 299.0
Đang xử lý bài hát 260/1744: 309.0
Đang xử lý bài hát 270/1744: 319.0
Đang xử lý bài hát 280/1744: 329.0
Đang xử lý bài hát 290/1744: 341.0
Đ

In [9]:
# Tải lại mô hình và scaler để sử dụng
model_valence = joblib.load('/media/haphuthinh/Data/Workspace/UIT/DO_AN_2/MusicEmotionDetection/TrainedModel/random_forest_regressor/random_forest_valence_model.pkl')
model_arousal = joblib.load('/media/haphuthinh/Data/Workspace/UIT/DO_AN_2/MusicEmotionDetection/TrainedModel/random_forest_regressor/random_forest_arousal_model.pkl')
scaler = joblib.load('/media/haphuthinh/Data/Workspace/UIT/DO_AN_2/MusicEmotionDetection/TrainedModel/random_forest_regressor/random_forest_scaler.pkl')

mp3_path = "/media/haphuthinh/Data/Workspace/UIT/DO_AN_2/MusicEmotionDetection/TestSong/BuonCuaAnh-KICMDatGMasew-9213751.mp3"
if os.path.exists(mp3_path):
    print("Đang phân tích bài hát...")
    predicted_emotion, valence, arousal = predict_emotion(mp3_path, model_valence, model_arousal, scaler)
    print(f"Cảm xúc dự đoán của bài hát là: {predicted_emotion} (valence: {valence}, arousal: {arousal})")
else:
    print("Đường dẫn không hợp lệ, vui lòng kiểm tra lại!")

Đang phân tích bài hát...


/tmp/ipykernel_22193/1900758238.py:8: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(y=y, sr=sr)[0]                         # 1 đặc trưng


Cảm xúc dự đoán của bài hát là: SAD (valence: 4.787000000000002, arousal: 4.699900000000002)
